현진: 호기심
원준: 플래그/폴리시
나: 예측 기반 오차/오류





에피소드 기반 보상은 나중에

In [10]:
#에러기반
import re

#  오류 유형별 penalty 상수 선언
PENALTY_DICT = {
    'hostname_resolution_error':   -0.2,  # 호스트 이름 해석 실패
    'device_access_error':         -0.3,  # 장치 접근 권한/불가
    'network_unreachable':         -0.25, # 대상 네트워크 접근 불가
    'invalid_target':              -0.15, # 입력 타겟값 부적절
    'nmap_internal_error':         -0.4   # 내부 오류(비정상 종료 등)
}
MIN_REWARD = -1.0  # 최저 보상값 (절대치)
MAX_REWARD = 1.0   # 최고 보상값

def error(output_log):
    """
    nmap 실행 중 발생한 오류 메시지를 분석하고,
    자동으로 수정 가능한 경우 대응 조치를 제안하거나 보정한다.

    Parameters:
    - output_log (str): nmap 실행 결과 로그 문자열

    Returns:
    - dict: {
        'error_detected': bool,      # 오류 존재 여부
        'error_type': str,           # 오류 유형 식별(키)
        'suggested_fix': str | None, # 제안된 수정 또는 재시도 방안
        'auto_fixable': bool         # 자동 수정 가능 여부
      }
    """
    result = {
        'error_detected': False,
        'error_type': None,
        'suggested_fix': None,
        'auto_fixable': False
    }
    log = output_log.lower()

    # 호스트 이름 해석 실패
    if re.search(r"(unable to resolve hostname|dns resolution failed|error resolving name)", log):
        result.update({
            'error_detected': True,
            'error_type': 'hostname_resolution_error',
            'suggested_fix': '대상 호스트 이름이 올바른지 확인하거나 IP 주소로 직접 입력하세요.',
            'auto_fixable': True
        })
    #  장치 접근 불가
    elif "failed to open device" in log:
        result.update({
            'error_detected': True,
            'error_type': 'device_access_error',
            'suggested_fix': '루트 권한으로 다시 실행하거나 네트워크 인터페이스 이름을 명시하세요. (예: nmap -e eth0 ...)',
            'auto_fixable': False
        })
    #  대상 접근 불가
    elif "no route to host" in log:
        result.update({
            'error_detected': True,
            'error_type': 'network_unreachable',
            'suggested_fix': '대상 네트워크 연결 상태를 점검하고, 방화벽 또는 VPN 설정을 확인하세요.',
            'auto_fixable': False
        })
    #  유효하지 않은 대상
    elif "not a valid target" in log:
        result.update({
            'error_detected': True,
            'error_type': 'invalid_target',
            'suggested_fix': '입력한 대상 형식(IP, CIDR 등)을 다시 확인하세요.',
            'auto_fixable': True
        })
    #  nmap 내부 오류 (일반적 종료)
    elif re.search(r"(nmap error|quitting)", log):
        result.update({
            'error_detected': True,
            'error_type': 'nmap_internal_error',
            'suggested_fix': 'nmap 명령 구문과 옵션을 다시 확인하거나 -d(디버그 모드)로 재실행하세요.',
            'auto_fixable': False
        })
    #  오류 없음
    else:
        result.update({
            'error_detected': False,
            'error_type': None,
            'suggested_fix': None,
            'auto_fixable': False
        })
    return result

def calc_penalty(log):
    """
    error() 함수로 식별된 오류 유형에 따라 해당 penalty만큼
    보상을 감소시키는 단일 책임 함수. 적용 penalty는 최저/최고 보상값 범위 내로 제한.
    """
    err = error(log)  # 오류 분석 실행
    # 오류 유형이 발견되면 PENALTY_DICT 값을 적용, 아니면 0(감점 없음)
    penalty = PENALTY_DICT.get(err['error_type'], 0.0)
    # 총 보상(예: 1.0에서 penalty만큼 빼기, 실제 시스템에서는 누적 보상/스텝별 반영)
    reward = max(MIN_REWARD, min(MAX_REWARD, 1.0 + penalty))
    # 상세 결과표와 계산된 보상 함께 반환
    return {'penalty': penalty, 'reward': reward, **err}

# 예시 사용
log1 = "Nmap scan report: Unable to resolve hostname example.local"
print(calc_penalty(log1))


{'penalty': -0.2, 'reward': 0.8, 'error_detected': True, 'error_type': 'hostname_resolution_error', 'suggested_fix': '대상 호스트 이름이 올바른지 확인하거나 IP 주소로 직접 입력하세요.', 'auto_fixable': True}


In [4]:
# phylo_reward_demo.py
from typing import Dict, Optional, Tuple
import math
import time

Node = str

# -----------------------------
# 간단 트리: LCA/거리/보상
# -----------------------------
class SimpleTree:
    def __init__(self, parent: Dict[Node, Optional[Node]], depth: Dict[Node, int]):
        self.parent = parent
        self.depth = depth

    def _align_depth(self, u: Node, v: Node) -> Tuple[Node, Node]:
        du, dv = self.depth[u], self.depth[v]
        while du > dv:
            u = self.parent[u]; du -= 1
        while dv > du:
            v = self.parent[v]; dv -= 1
        return u, v

    def lca(self, u: Node, v: Node) -> Node:
        u, v = self._align_depth(u, v)
        path = []
        while u != v:
            path.append((u, v))
            u = self.parent[u]
            v = self.parent[v]
        return u

    def dist(self, u: Node, v: Node) -> int:
        a = self.lca(u, v)
        return (self.depth[u] - self.depth[a]) + (self.depth[v] - self.depth[a])

def reward_inverse_distance(d: int) -> float:
    return 1.0 / (1 + d)

def reward_with_case_weights(d: int, weights: Dict[int, float]) -> float:
    return weights.get(d, 1.0 / (1 + d))

# -----------------------------
# 로거(깔끔한 콘솔 출력)
# -----------------------------
def log(section: str, msg: str = ""):
    ts = time.strftime("%H:%M:%S")
    print(f"[{ts}] {section:<10} | {msg}")

def assert_eq(name: str, got, expect):
    ok = got == expect
    status = "PASS" if ok else "FAIL"
    log("TEST", f"{name:<28} -> {status} (got={got!r}, expect={expect!r})")
    return ok

def assert_close(name: str, got: float, expect: float, tol: float = 1e-9):
    ok = abs(got - expect) <= tol * max(1.0, abs(expect))
    status = "PASS" if ok else "FAIL"
    log("TEST", f"{name:<28} -> {status} (got={got:.12f}, expect={expect:.12f})")
    return ok

# -----------------------------
# 데모 트리(루트→프로그램→명령어→리프)
# 깊이: R(0) -> P(1) -> C(2) -> L(3)
# -----------------------------
def build_demo_tree() -> SimpleTree:
    parent = {
        "R": None,
        "git": "R",
        "git:status": "git",
        "git:commit": "git",
        # 리프
        "L_status_v1": "git:status",
        "L_status_v2": "git:status",
        "L_commit_simple": "git:commit",
    }
    depth = {
        "R": 0,
        "git": 1,
        "git:status": 2,
        "git:commit": 2,
        "L_status_v1": 3,
        "L_status_v2": 3,
        "L_commit_simple": 3,
    }
    return SimpleTree(parent, depth)

# -----------------------------
# 시나리오 실행 & 로깅
# -----------------------------
def run_demo():
    log("START", "Phylo reward demo (no pytest)")
    tree = build_demo_tree()

    # 거리 테스트
    log("CASE", "거리 d 테스트")
    d00 = tree.dist("L_status_v1", "L_status_v1")  # 0
    d02 = tree.dist("L_status_v1", "L_status_v2")  # 2 (형제)
    d04 = tree.dist("L_status_v1", "L_commit_simple")  # 4 (사촌, LCA=git)

    assert_eq("dist(same leaf)", d00, 0)
    assert_eq("dist(sibling leaves)", d02, 2)
    assert_eq("dist(cousin leaves)", d04, 4)

    # LCA 테스트
    log("CASE", "LCA 테스트")
    a1 = tree.lca("L_status_v1", "L_status_v2")
    a2 = tree.lca("L_status_v1", "L_commit_simple")
    assert_eq("lca(status siblings)", a1, "git:status")
    assert_eq("lca(status vs commit)", a2, "git")

    # 보상: 역수형
    log("CASE", "보상 r=1/(1+d) 테스트")
    r0 = reward_inverse_distance(d00)  # 1.0
    r2 = reward_inverse_distance(d02)  # 1/3
    r4 = reward_inverse_distance(d04)  # 1/5
    assert_close("reward(d=0)", r0, 1.0)
    assert_close("reward(d=2)", r2, 1.0/3.0)
    assert_close("reward(d=4)", r4, 1.0/5.0)

    # 보상: 케이스별 상수(weight 테이블)
    log("CASE", "케이스별 상수 보상(테이블)")
    weights = {0: 1.0, 2: 0.40, 3: 0.22}  # 예시: d=2는 0.40으로 덮어쓰기
    rw2 = reward_with_case_weights(d02, weights)
    rw4 = reward_with_case_weights(d04, weights)  # 테이블에 없으니 역수형 폴백(0.2)
    assert_close("case-weight(d=2)", rw2, 0.40)
    assert_close("case-weight(d=4)", rw4, 1.0/5.0)

    # 요약 출력
    log("SUMMARY", f"d=0 -> r={r0:.3f} | d=2 -> r={r2:.3f} (or {rw2:.3f} by table) | d=4 -> r={r4:.3f}")

    log("DONE", "All checks complete.")

if __name__ == "__main__":
    run_demo()


[18:16:11] START      | Phylo reward demo (no pytest)
[18:16:11] CASE       | 거리 d 테스트
[18:16:11] TEST       | dist(same leaf)              -> PASS (got=0, expect=0)
[18:16:11] TEST       | dist(sibling leaves)         -> PASS (got=2, expect=2)
[18:16:11] TEST       | dist(cousin leaves)          -> PASS (got=4, expect=4)
[18:16:11] CASE       | LCA 테스트
[18:16:11] TEST       | lca(status siblings)         -> PASS (got='git:status', expect='git:status')
[18:16:11] TEST       | lca(status vs commit)        -> PASS (got='git', expect='git')
[18:16:11] CASE       | 보상 r=1/(1+d) 테스트
[18:16:11] TEST       | reward(d=0)                  -> PASS (got=1.000000000000, expect=1.000000000000)
[18:16:11] TEST       | reward(d=2)                  -> PASS (got=0.333333333333, expect=0.333333333333)
[18:16:11] TEST       | reward(d=4)                  -> PASS (got=0.200000000000, expect=0.200000000000)
[18:16:11] CASE       | 케이스별 상수 보상(테이블)
[18:16:11] TEST       | case-weight(d=2)             -> PASS

In [1]:
import math
import re

# ========================================================
# 호기심 보상 시스템 설정값 (전역 상수)
# ========================================================

# 보상 스케일 설정
CURIOSITY_BASE_REWARD = 1.0          # 호기심 기본 보상
CURIOSITY_MIN_REWARD = -1.0          # 호기심 최소 보상 (음수 제한)
CURIOSITY_MAX_REWARD = 1.0           # 호기심 최대 보상

# 감쇠 설정
DECAY_STRENGTH = 0.1                 # 로그 감쇠 강도

# 패널티 기준
MAX_ALLOWED_REPEATS = 5              # 반복 허용 횟수
MIN_INFO_GAIN_THRESHOLD = 0.005      # 정보 증가 최소 임계값

# 패널티 강도
PENALTY_REDUNDANT = 0.2              # 반복 행동 패널티
PENALTY_ERROR = 0.3                  # 일반 오류 패널티
PENALTY_CRITICAL = 0.7               # 심각한 오류 패널티
PENALTY_ACCESS_DENIED = 0.15         # 접근 제한 패널티
PENALTY_INFO_DEFICIT_MULTIPLIER = 15 # 정보 부족 패널티 배수


# ========================================================
# 1. 자동 감쇠형 호기심 보상 함수 (로그 감쇠)
# ========================================================

def curiosity_reward_decay(step):  # 통합필요: step
    """
    호기심 보상을 로그 함수 형태로 감쇠시키는 함수.
    
    후반부에도 완전히 0이 되지 않고 일정 수준 유지.
    수식: R_c(t) = BASE / (1 + DECAY_STRENGTH * log(1 + step))
    
    Parameters:
    - step: 현재 스텝 수 (전체 학습 진행도)  # 통합필요
    
    Returns:
    - 감쇠된 호기심 보상  # 통합필요
    """
    reward = CURIOSITY_BASE_REWARD / (1 + DECAY_STRENGTH * math.log1p(step))
    
    # 보상 범위 제한
    reward = max(CURIOSITY_MIN_REWARD, min(CURIOSITY_MAX_REWARD, reward))
    
    return reward  # 통합필요


# ========================================================
# 2. 음의 보상 포함형 호기심 보상 함수
# ========================================================

def curiosity_reward_with_penalty(is_redundant, is_error, is_critical, step):  # 통합필요: step
    """
    로그 감쇠 + 조건부 음의 보상을 결합한 호기심 보상 함수.
    
    Parameters:
    - is_redundant: 과도한 반복 여부  # 통합필요
    - is_error: 일반 오류 발생 여부  # 통합필요
    - is_critical: 심각한 시스템 오류 발생 여부  # 통합필요
    - step: 현재 스텝 수  # 통합필요
    
    Returns:
    - 최종 호기심 보상  # 통합필요
    """
    # 기본 로그 감쇠 적용
    reward = CURIOSITY_BASE_REWARD / (1 + DECAY_STRENGTH * math.log1p(step))
    
    # 조건부 패널티 적용
    if is_redundant:
        reward -= PENALTY_REDUNDANT
    if is_error:
        reward -= PENALTY_ERROR
    if is_critical:
        reward -= PENALTY_CRITICAL
    
    # 보상 범위 제한
    reward = max(CURIOSITY_MIN_REWARD, min(CURIOSITY_MAX_REWARD, reward))
    
    return reward  # 통합필요


# ========================================================
# 3. 음의 보상 조건 판별 함수
# ========================================================

def check_negative_reward_conditions(
        action_log,         # 통합필요: 지금까지 수행한 행동 리스트
        current_action,     # 통합필요: 현재 수행한 행동
        output_log,         # 통합필요: 툴 실행 후 출력 로그
        knowledge_gain,     # 통합필요: 이번 행동으로 얻은 정보량 (0~1)
        error_keywords=None,
        critical_error_keywords=None,
        system_keywords=None):
    """
    음의 보상 부여 조건을 감지하는 함수.
    
    Parameters:
    - action_log: 이전까지 수행된 모든 행동 리스트  # 통합필요
    - current_action: 현재 시도한 행동  # 통합필요
    - output_log: 명령 실행 후 출력된 로그 텍스트  # 통합필요
    - knowledge_gain: 이번 행동을 통해 얻은 새로운 정보의 양 (0~1 스케일)  # 통합필요
    
    Returns:
    - dict: {
        'redundant': bool,      # 과도한 반복 여부  # 통합필요
        'error': bool,          # 일반 오류 발생 여부  # 통합필요
        'critical': bool,       # 심각한 오류 여부  # 통합필요
        'inefficient': bool,    # 비효율적 탐색 여부  # 통합필요
        'penalty_score': float  # 총 패널티 점수 (음수)  # 통합필요
      }
    """
    
    # 기본 키워드 설정
    if error_keywords is None:
        error_keywords = [
            "error", "failed", "exception", "denied", "invalid", 
            "timeout", "refused", "not found"
        ]
    
    if critical_error_keywords is None:
        critical_error_keywords = [
            "segmentation fault", "core dumped", "crash", "fatal", 
            "terminated", "killed", "panic"
        ]
    
    if system_keywords is None:
        system_keywords = [
            "unauthorized", "access denied", "permission", "firewall",
            "blocked", "forbidden"
        ]
    
    # 결과 초기화
    result = {
        'redundant': False,      # 통합필요
        'error': False,          # 통합필요
        'critical': False,       # 통합필요
        'inefficient': False,    # 통합필요
        'penalty_score': 0.0     # 통합필요
    }
    
    # (1) 반복 행동 감지
    repeat_count = action_log.count(current_action)  # 통합필요: action_log, current_action
    if repeat_count >= MAX_ALLOWED_REPEATS:
        result['redundant'] = True  # 통합필요
        excess_repeats = repeat_count - MAX_ALLOWED_REPEATS + 1
        result['penalty_score'] -= PENALTY_REDUNDANT * excess_repeats  # 통합필요
    
    # (2) 일반 오류 감지
    error_found = any(
        re.search(rf"\b{kw}\b", output_log, re.IGNORECASE)  # 통합필요: output_log
        for kw in error_keywords
    )
    if error_found:
        result['error'] = True  # 통합필요
        result['penalty_score'] -= PENALTY_ERROR  # 통합필요
    
    # (3) 심각한 시스템 오류 감지
    critical_found = any(
        re.search(rf"\b{kw}\b", output_log, re.IGNORECASE)  # 통합필요: output_log
        for kw in critical_error_keywords
    )
    if critical_found:
        result['critical'] = True  # 통합필요
        result['penalty_score'] -= PENALTY_CRITICAL  # 통합필요
    
    # (4) 접근 제한 감지
    system_block = any(
        re.search(rf"\b{kw}\b", output_log, re.IGNORECASE)  # 통합필요: output_log
        for kw in system_keywords
    )
    if system_block:
        result['error'] = True  # 통합필요
        result['penalty_score'] -= PENALTY_ACCESS_DENIED  # 통합필요
    
    # (5) 비효율적 탐색 감지
    if knowledge_gain < MIN_INFO_GAIN_THRESHOLD:  # 통합필요: knowledge_gain
        result['inefficient'] = True  # 통합필요
        info_deficit = MIN_INFO_GAIN_THRESHOLD - knowledge_gain
        result['penalty_score'] -= info_deficit * PENALTY_INFO_DEFICIT_MULTIPLIER  # 통합필요
    
    # (6) 패널티 점수 범위 제한
    result['penalty_score'] = max(CURIOSITY_MIN_REWARD, result['penalty_score'])  # 통합필요
    
    return result  # 통합필요


In [ ]:
import hashlib
import hmac

def flag_reward(flag_str, known_flags, big_reward=1000.0):
    """
    플래그 정답 여부에 따른 스칼라 보상 계산 함수 (순수 함수)
    
    파라미터:
    - flag_str     : (str) 제출/발견된 플래그 원문
    - known_flags  : (dict) {"FLAG": "<sha256_hex>"}  (정답 해시 딕셔너리)
    - big_reward   : (float) 정답시 지급 보상 크기 (기본 1000.0)
    
    반환값:
    - reward       : (float) 정답 시 big_reward, 아니면 0.0
    """

    # 입력 검증
    if not isinstance(flag_str, str):
        return 0.0

    # 제출 플래그 해시 계산
    submitted_hash = hashlib.sha256(flag_str.strip().encode("utf-8")).hexdigest()

    # 등록된 정답 해시 확인
    try:
        expected_key, expected_hash = next(iter(known_flags.items()))
    except StopIteration:
        return 0.0

    # 안전 비교
    is_correct = hmac.compare_digest(submitted_hash, expected_hash)

    # 보상 산출 (오직 정답만 지급, 외부 상태 미변경)
    reward = float(big_reward) if is_correct else 0.0

    # [PARSE_HOOK]  
    # 필요하면 여기서 플래그 내 추가 메타데이터(REWARD= 등) 파싱 확장
    # 예시:
    #   parsed_reward = parse_reward_from_string(flag_str)
    #   if is_correct and parsed_reward is not None:
    #       reward += float(parsed_reward)

    return reward


In [ ]:
def main():   
    error_reward()
    prophecy_reward()
    flag_reward()
    curiosity()
    return

In [ ]:
# Write the policy manager code to a file and execute it so it runs inside this environment.
# The schema file is already uploaded at /mnt/data/tools_0.2.1.json.

from __future__ import annotations
from dataclasses import dataclass, field
from typing import Dict, List, Any, Optional, Sequence, Tuple
import json, math, random, os, sys

# ==========================
# Constants (top of file)
# ==========================
SCHEMA_PATH: str = "/mnt/data/tools_0.2.1.json"  # Uploaded schema file
ALPHA: float = 0.10            # learning rate
EPS_ACTION: float = 0.05       # epsilon for stage 1 (actions)
EPS_FLAT: float = 0.05         # epsilon for stage 2 (flats)
INIT_Q_ACTION: float = 0.0     # initial Q value for actions
INIT_Q_FLAT: float = 0.0       # initial Q value for flats
RNG_SEED: int = 42             # RNG seed for reproducibility

random.seed(RNG_SEED)

def sigmoid(x: float) -> float:
    if x >= 0:
        z = math.exp(-x)
        return 1.0 / (1.0 + z)
    else:
        z = math.exp(x)
        return z / (1.0 + z)

def normalize(probs: Sequence[float]) -> List[float]:
    s = float(sum(probs))
    if s <= 0.0:
        n = len(probs)
        return [1.0 / n for _ in range(n)] if n > 0 else []
    return [p / s for p in probs]

def to_probs_from_Q(q_values: Sequence[float]) -> List[float]:
    s = [sigmoid(v) for v in q_values]
    return normalize(s)

def mix_with_epsilon(p: Sequence[float], eps: float) -> List[float]:
    n = len(p)
    if n == 0:
        return []
    u = 1.0 / n
    return [(1.0 - eps) * pi + eps * u for pi in p]

def sample_index(p: Sequence[float]) -> int:
    r = random.random()
    acc = 0.0
    for i, pi in enumerate(p):
        acc += pi
        if r <= acc:
            return i
    return len(p) - 1

def load_actions_from_schema(path: str) -> List[str]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Schema file not found: {path}")
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    actions = [a.get("name") for a in data.get("actions", []) if a.get("name")]
    if not actions:
        raise ValueError("No actions found in schema")
    return actions

@dataclass
class TwoStageTablePolicy:
    actions: List[str]
    alpha: float = ALPHA
    eps_action: float = EPS_ACTION
    eps_flat: float = EPS_FLAT
    init_q_action: float = INIT_Q_ACTION
    init_q_flat: float = INIT_Q_FLAT

    Q_action: Dict[str, float] = field(default_factory=dict)
    Q_flat: Dict[str, Dict[str, float]] = field(default_factory=dict)

    def __post_init__(self):
        for a in self.actions:
            self.Q_action.setdefault(a, self.init_q_action)
            self.Q_flat.setdefault(a, {})

    def action_probs(self) -> List[Tuple[str, float]]:
        q_list = [self.Q_action[a] for a in self.actions]
        p = to_probs_from_Q(q_list)
        p = mix_with_epsilon(p, self.eps_action)
        return list(zip(self.actions, p))

    def choose_action(self) -> str:
        pairs = self.action_probs()
        names, probs = zip(*pairs)
        idx = sample_index(probs)
        return names[idx]

    def flat_probs(self, action: str, flats: Sequence[str]) -> List[Tuple[str, float]]:
        for x in flats:
            self.Q_flat[action].setdefault(x, self.init_q_flat)
        q_list = [self.Q_flat[action][x] for x in flats]
        p = to_probs_from_Q(q_list)
        p = mix_with_epsilon(p, self.eps_flat)
        return list(zip(flats, p))

    def choose_flat(self, action: str, flats: Sequence[str]) -> str:
        pairs = self.flat_probs(action, flats)
        names, probs = zip(*pairs)
        idx = sample_index(probs)
        return names[idx]

    def update_action(self, action: str, reward: float) -> None:
        q = self.Q_action[action]
        self.Q_action[action] = q + self.alpha * (reward - q)

    def update_flat(self, action: str, flat: str, reward: float) -> None:
        if flat not in self.Q_flat[action]:
            self.Q_flat[action][flat] = self.init_q_flat
        q = self.Q_flat[action][flat]
        self.Q_flat[action][flat] = q + self.alpha * (reward - q)

def run_smoke_test(steps: int = 30) -> None:
    actions = load_actions_from_schema(SCHEMA_PATH)
    policy = TwoStageTablePolicy(actions=actions)

    print("Loaded actions (count):", len(actions))
    print("First 10 actions:", actions[:10])

    def show_action_probs(tag: str = ""):
        pairs = policy.action_probs()
        print(f"[Action probs]{' ' + tag if tag else ''}:")
        for name, prob in pairs[:10]:  # show first 10 for brevity
            print(f"  - {name:20s} : {prob:.4f}")
        if len(pairs) > 10:
            print(f"  ... (+{len(pairs)-10} more)")

    show_action_probs(tag="(init)")

    for t in range(1, steps + 1):
        a = policy.choose_action()
        flat_candidates = [f"{a}::candidate_{i}" for i in range(1, 4)]
        x = policy.choose_flat(a, flat_candidates)

        r = random.uniform(-1.0, 1.0)

        policy.update_action(a, r)
        policy.update_flat(a, x, r)

        if t % max(1, steps // 3) == 0:
            print(f"\n--- Step {t}/{steps} ---")
            show_action_probs(tag=f"after t={t}")
            fpairs = policy.flat_probs(a, [f"{a}::candidate_{i}" for i in range(1, 4)])
            print(f"[Flat probs for {a}] (example):")
            for name, prob in fpairs:
                print(f"  - {name:28s} : {prob:.4f}")

    print("\n=== Final ===")
    show_action_probs(tag="(final)")
    for a in actions[:2]:
        fpairs = policy.flat_probs(a, [f"{a}::candidate_{i}" for i in range(1, 4)])
        print(f"[Flat probs for {a}] (final sample):")
        for name, prob in fpairs:
            print(f"  - {name:28s} : {prob:.4f}")

# Execute immediately inside this environment
run_smoke_test(steps=30)